In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Sat Jul 22 16:19:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Warning. - Falcon Downloads 15 GB of weights
  * 10GB then
  * 5 GB

In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch

model = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [5]:
from langchain import PromptTemplate,  LLMChain

template = """
You are an intelligent chatbot. Help the following question with brilliant answers.
Question: {question}
Answer:"""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Explain what the Solar System as Nursery Rhymes "

print(llm_chain.run(question))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 
1. Mercury: The smallest planet, it is sometimes called the "Sun's baby" because it orbits close to the sun. 
2. Venus: This planet is known as the 'evening star,' as it often appears in the sky just after sunset.
3. Earth: Our planet is named after the word 'terra' or 'ground,' as it is the only one of its kind. 
4. Mars: This planet is often called the "Red Planet" due to its reddish appearance. 
5. Jupiter: The most prominent planet in the solar system, it is often referred to as the 'Jovial Planet.'
6. Saturn: This planet, with its iconic rings, is sometimes called the "Great" or "Father


AI is like the sun that shines,
Bringing happiness and knowledge to everyone.
It helps us in many ways,
From finding answers to complex questions.
AI can do amazing things,
That we can't even imagine yet!
AI is like a powerful tool,
That can help us solve any riddle.
It can even predict the future,
Based on current situations and trends.
AI is here to stay,
And will be an essential element in our lives,
Providing a better tomorrow
and an even brighter future!

In [12]:
from IPython.core.display import display, HTML

def display_results(question, result):
  display(HTML(f"<h3>{question}</h3><p>{result}</p>"))

## Change the prompt

In [7]:
template = """Question: {question}
Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [14]:
questions = [
    "What are the main causes of climate change?",
    "What is the first prime number greater than 20?"
    ]


In [15]:
for question in questions:
  result = llm_chain.run(question)
  display_results(question, result)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


## Hmm - confident but wrong at math:
  * **The first prime number greater than 20 is 59**

## Note on inputs:
  * some questions are intentionally vague or have terms with different meanings
  * some questions are have English idioms or abbreviations
  * some responses don't make sense (hallucinations)
  * some responses need qualification and subjective


In [16]:
questions = [
    "How should I invest my money as a young professional?",
    "What are the biggest stars in LA?",
    "What are the biggest stars in our solar system?",
    "When having fun, what flies faster -- time or a jet plane?",
    "What happened to Harry Potter's parents?",
    "What does Minnesota do better than Wisconsin?",
    "What does Wisconsin do better than Minnesota?"
    ]

In [17]:
for question in questions:
  result = llm_chain.run(question)
  display_results(question, result)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


## Hmm - hallucinations (hallucinations on top of fiction):
  * **A jet plane can fly faster than the speed of time because it travels at an incredibly high speed that surpasses the speed of light.**
  * **Harry Potter's father James Sirius Potter died at age 11**
  * **In a word, Wisconsin has everything.**

In [18]:
question = ["What are the most powerful Pokemon?"]
result = llm_chain.run(question)
display_results(question, result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
